*Portions of the Python code, including data cleaning, data preparation, and segments of the regression analysis, have been adapted from the Stata code provided in Kaboski and Townsend's paper:*

Kaboski, Joseph P., and Robert M. Townsend. "The Impact of Credit on Village Economies." *American Economic Journal: Applied Economics* 4, no. 2 (2012): 98–133. https://doi.org/10.1257/app.4.2.98.

## Data Cleaning

In [ ]:
import warnings
import logging
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import gaussian_kde, chi2
from scipy.signal import find_peaks
import statsmodels.api as sm
from sklearn.model_selection import train_test_split, KFold, GridSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LassoCV, Ridge, LogisticRegression, lasso_path
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, BaggingRegressor
from sklearn.tree import DecisionTreeRegressor, plot_tree
from sklearn.neighbors import NearestNeighbors
from lightgbm import LGBMRegressor
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.callbacks import EarlyStopping
from econml.dml import CausalForestDML
from linearmodels.panel import PanelOLS
from graphviz import Digraph

# Suppress warnings
warnings.filterwarnings('ignore')

# Load and preprocess the data
df = pd.read_stata('AnnualData_ShortSample.dta')
df.sort_values('case_id', inplace=True)
df['tinv'] = df[['hinv', 'finv', 'pinv', 'binv', 'bafdn', 'sinv', 'safdn', 'lvcn', 'lvnn', 'lvdep']].sum(axis=1)
df['twage'] = df['frmwage'] + df['shrwage'] + df['buswage']
df.drop(columns=['frmwage', 'shrwage', 'buswage'], inplace=True)
df['farm'] = ((df['och'] > -5) & (df['och'] <= 15)).astype(int)
df['tbinv'] = df[['pinv', 'binv', 'bafdn', 'sinv', 'safdn']].sum(axis=1)
df['age2h'] = df['ageh'] ** 2
df['gassd2'] = df['gassd'] ** 2
df['lgassd'] = np.log(df['gassd'])
df['dlgassd1'] = df.groupby('case_id')['gassd'].transform(lambda x: np.log(x.shift(-1) / x))
df['dnetinc'] = df.groupby('case_id')['netinc'].transform(lambda x: np.log(x.shift(-1) / x))
df['invHH'] = 1 / df['vHH']
df.loc[df['year'] == 1, 'invHH'] = df.groupby('case_id')['invHH'].transform(lambda x: x.shift(-1))
df['invHHl'] = df.groupby('case_id')['vHH'].transform(lambda x: 1 / x.shift(1))
df['vfstl'] = df.groupby('case_id')['vfst'].transform(lambda x: x.shift(1))
df['bsnew'] = df['bnew'] + df['snew']

# Specify columns to keep
columns_to_keep = [
    'case_id', 'year', 'frmpro', 'buspro', 'wageinc', 'riceinc', 'cropinc', 'liveinc',
    'educ', 'invHHl', 'vfstl', 'invHH', 'changwat', 'amphoe', 'tambon', 'village',
    'dnetinc', 'dlgassd1', 'farm', 'netinc', 'bsnew', 'tc', 'educ', 'grain', 'milk',
    'meat', 'alch1', 'alch2', 'fuel', 'tobac', 'cerem', 'houserep', 'vehicrep',
    'clothes', 'mealaway', 'ageh', 'madult', 'fadult', 'kids', 'maleh', 'finv',
    'tbinv', 'hinv', 'frtexp', 'netinc', 'lac', 'newst', 'vfst', 'infst', 'baacst',
    'cbst', 'const', 'edust', 'agst', 'hhast', 'busst', 'frtst', 'age2h', 'educh',
    'gassd', 'gassd2', 'lgassd', 'rst', 'defcr', 'twage'
]
df = df[columns_to_keep]

# Additional transformations
df['villageyear'] = (df['case_id'] // 1000).astype(int)
df['vfst'] = df['vfst'] / 10000
df['vfstl'] = df['vfstl'] / 10000
df.sort_values(by=['case_id', 'year'], inplace=True)
df['invHHim'] = np.where(df['year'] == 6, df['invHH'], np.nan)
df['invHHi'] = df.groupby('case_id')['invHHim'].transform('mean')
df['vHHi'] = 1 / df['invHHi']
df = df[(df['vHHi'] <= 250) & (df['vHHi'] >= 50)]
df['invHHpvf'] = df['invHHi'] * (df['year'] > 5)
df['invHHtvf1'] = df['invHHi'] * (df['year'] == 6)
df['invHHtvf2'] = df['invHHi'] * (df['year'] == 7)
df['vfstf'] = df['vfst'] * (df['maleh'] == 0)
df['invHHtvf1f'] = df['invHHtvf1'] * (df['maleh'] == 0)
df['invHHtvf2f'] = df['invHHtvf2'] * (df['maleh'] == 0)
df['vfstlf'] = df['vfstl'] * (df['maleh'] == 0)
df['caseid'] = df['case_id'].astype(float)
df.drop(columns=['case_id'], inplace=True)
df['tbinvp'] = df['tbinv'] > 0
df['finvp'] = df['finv'] > 0
df['defcrp'] = df['defcr'] > 0
df = df[df['year'] < 8]

## Summary Statistics

### General Summary

In [ ]:
# Create a mapping for variable names
variable_names = {
    'newst': "New Short-Term Credit",
    'vfst_post_program': "Village Fund Credit",
    'ageh': "Age of Household's Head",
    'educh': "Years of Education of Household's Head"
}

# Create 'vfst_post_program' variable
df['vfst_post_program'] = df['vfst'].where(df['year'].isin([6, 7]), other=None)

# Calculate the mean for specified variables grouped by 'caseid'
collapsed_df = df.groupby('caseid')[['newst', 'vfst_post_program', 'ageh', 'educh']].mean()

# Scale 'vfst_post_program' for display purposes
collapsed_df['vfst_post_program'] *= 10000

# Rename columns for readability
collapsed_df.rename(columns=variable_names, inplace=True)

# Get summary statistics
summary_stats_interest = collapsed_df.describe()

# Calculate cross-sectional standard deviation
cross_sectional_std = collapsed_df.std()

# Transpose the summary statistics table and reorder rows
summary_stats_transposed = summary_stats_interest.T.reindex([
    "New Short-Term Credit", 
    "Village Fund Credit",  
    "Age of Household's Head", 
    "Years of Education of Household's Head"
])

# Add cross-sectional standard deviation
summary_stats_transposed['Cross-Sectional Standard Deviation'] = cross_sectional_std.reindex([
    "New Short-Term Credit", 
    "Village Fund Credit",  
    "Age of Household's Head", 
    "Years of Education of Household's Head"
])

# Calculate counts and proportions for 'farm' dummy variable
farm_counts = df['farm'].value_counts().rename({0: 'Non-Farm', 1: 'Farm'})
farm_proportions = df['farm'].value_counts(normalize=True).rename({0: 'Non-Farm', 1: 'Farm'}) * 100

# Create a summary DataFrame for 'farm'
farm_summary = pd.DataFrame({
    'Count': farm_counts,
    'Proportion (%)': farm_proportions
})

### Age of Household's Head

In [ ]:
# Set seaborn style
sns.set_style("whitegrid")

# Extract 'ageh' data and drop NaN values
ageh_data = df['ageh'].dropna()

# Plot histogram of age with percentages
plt.figure(figsize=(10, 6))
sns.histplot(
    ageh_data,
    bins=20,
    kde=False,
    color='#4c72b0',
    edgecolor='white',
    stat="percent"
)

# Customize plot
plt.title("Histogram of Age of Household's Head", fontsize=18, weight='bold', pad=20)
plt.xlabel("Age of Household's Head", fontsize=14)
plt.ylabel("Percentage (%)", fontsize=14)
sns.despine()
plt.grid(True, linestyle='--', alpha=0.5)
plt.tight_layout()

# Show the plot
plt.show()

In [ ]:
# Set seaborn style
sns.set_style("whitegrid")

# Create density plot for 'ageh'
plt.figure(figsize=(10, 6))
sns.kdeplot(df['ageh'].dropna(), shade=True, color='#4c72b0', lw=2)

# Customize plot
plt.title("Density Plot of Age of Household's Head", fontsize=18, weight='bold', pad=20)
plt.xlabel("Age of Household's Head", fontsize=14)
plt.ylabel("Density", fontsize=14)
sns.despine()
plt.grid(True, linestyle='--', alpha=0.5)
plt.tight_layout()

# Show the plot
plt.show()

In [ ]:
# Extract 'ageh' data and drop NaN values
ageh_data = df['ageh'].dropna()

# Perform Gaussian Kernel Density Estimation
kde = gaussian_kde(ageh_data)
age_range = np.linspace(ageh_data.min(), ageh_data.max(), 1000)
density_values = kde(age_range)

# Find local minima
minima_indices = find_peaks(-density_values)[0]
minima_ages = age_range[minima_indices]

# Plot density with minima
plt.figure(figsize=(10, 6))
sns.set_style("whitegrid")
plt.plot(age_range, density_values, color='#4c72b0', lw=2, label='Density')
plt.scatter(minima_ages, density_values[minima_indices], color='#e76f51', zorder=5, s=100, label='Minima (Split Points)')

# Annotate minima
for i, age in enumerate(minima_ages):
    plt.text(age, density_values[minima_indices][i] + 0.002, f'{age:.2f}', fontsize=12, color='black', ha='center')

# Customize plot
plt.title("Density Plot of Age of Household's Head with Minima", fontsize=18, weight='bold', pad=20)
plt.xlabel("Age of Household's Head", fontsize=14)
plt.ylabel("Density", fontsize=14)
plt.grid(True, linestyle='--', alpha=0.5)
sns.despine()
plt.legend(fontsize=12)
plt.tight_layout()

# Show plot
plt.show()

# Print minima
print(f"Split points (minima) in the distribution: {minima_ages}")

In [ ]:
# Extract 'ageh' column and drop NaN values
ageh_data = df['ageh'].dropna()

# Perform Kernel Density Estimation
kde = gaussian_kde(ageh_data)
age_range = np.linspace(ageh_data.min(), ageh_data.max(), 1000)
density_values = kde(age_range)

# Identify local minima as split points
minima_indices = find_peaks(-density_values)[0]
minima_ages = age_range[minima_indices]

# Determine the split point
split_age = minima_ages[0]

# Categorize ages into 'Young' and 'Old'
df['age_group'] = np.where(df['ageh'] <= split_age, 'Young', 'Old')

# Compute and rename summary statistics
summary_stats = df.groupby('age_group')['ageh'].describe().rename(columns={
    'count': 'Count',
    'mean': 'Mean',
    'std': 'Std Dev',
    'min': 'Min',
    '25%': '25%',
    '50%': 'Median',
    '75%': '75%',
    'max': 'Max'
})
summary_stats.index.name = 'Age Group'

# Display the summary statistics
summary_stats.style.set_caption("Summary Statistics for Age Groups").format("{:.2f}")

### Years of Education of Household Head

In [ ]:
# Set seaborn style
sns.set_style("whitegrid")

# Extract 'educh' data and drop NaN values
educh_data = df['educh'].dropna()

# Plot histogram of years of education
plt.figure(figsize=(10, 6))
sns.histplot(
    educh_data,
    bins=20,
    kde=False,
    color='#4c72b0',
    edgecolor='white',
    stat="percent"
)

# Customize plot
plt.title("Histogram of Years of Education", fontsize=18, weight='bold', pad=20)
plt.xlabel("Years of Education", fontsize=14)
plt.ylabel("Percentage (%)", fontsize=14)
sns.despine()
plt.grid(True, linestyle='--', alpha=0.5)
plt.tight_layout()

# Show the plot
plt.show()

In [ ]:
# Define buckets for education levels
bins = [0, 4, 8, 16]
labels = ['Low (0-4 years)', 'Medium (5-8 years)', 'High (9-16 years)']

# Create 'education_bucket' column
df['education_bucket'] = pd.cut(df['educh'], bins=bins, labels=labels, include_lowest=True)

# Generate summary statistics for each bucket
summary_stats_buckets = df.groupby('education_bucket')['educh'].describe()

# Rename index and columns
summary_stats_buckets.rename(columns={
    'count': 'Count',
    'mean': 'Mean',
    'std': 'Std Dev',
    'min': 'Min',
    '25%': '25%',
    '50%': 'Median',
    '75%': '75%',
    'max': 'Max'
}, inplace=True)
summary_stats_buckets.index.name = 'Education Group'

# Display
summary_stats_buckets.style.set_caption("Summary Statistics for Education Buckets").format("{:.2f}")

In [ ]:
# Set seaborn style
sns.set_style("whitegrid")

# Plot histogram for education buckets with percentages
plt.figure(figsize=(10, 6))
sns.histplot(
    df['education_bucket'],
    color='#4c72b0',
    edgecolor='white',
    shrink=0.8,
    stat="percent"
)

# Customize plot
plt.title("Education Buckets", fontsize=18, weight='bold', pad=20)
plt.xlabel("Education Level Buckets", fontsize=14)
plt.ylabel("Percentage (%)", fontsize=14)
sns.despine()
plt.grid(True, linestyle='--', alpha=0.5)
plt.tight_layout()

# Show plot
plt.show()

### Farmer Status

In [ ]:
# Create 'farmer_status' column
df['farmer_status'] = np.where(df['farm'] == 1, 'Farmer', 'Non-Farmer')

# Summary statistics for farmers and non-farmers
summary_stats_farmers = df.groupby('farmer_status')['farm'].describe()

# Rename index and columns
summary_stats_farmers.rename(columns={
    'count': 'Count',
    'mean': 'Mean',
    'std': 'Std Dev',
    'min': 'Min',
    '25%': '25%',
    '50%': 'Median',
    '75%': '75%',
    'max': 'Max'
}, inplace=True)
summary_stats_farmers.index.name = 'Farmer Status'

# Display
summary_stats_farmers.style.set_caption("Summary Statistics for Farmers and Non-Farmers").format("{:.2f}")

In [ ]:
# Add 'farmer_status' column
df['farmer_status'] = np.where(df['farm'] == 1, 'Farmer', 'Non-Farmer')

# Set seaborn style
sns.set_style("whitegrid")

# Plot histogram for farm status
plt.figure(figsize=(10, 6))
sns.histplot(
    df['farmer_status'],
    color='#4c72b0',
    edgecolor='white',
    shrink=0.8,
    stat="percent"
)

# Customize plot
plt.title("Farm Status", fontsize=18, weight='bold', pad=20)
plt.xlabel("Farm Status (Farmer vs Non-Farmer)", fontsize=14)
plt.ylabel("Percentage (%)", fontsize=14)
sns.despine()
plt.grid(True, linestyle='--', alpha=0.5)
plt.tight_layout()

# Show plot
plt.show()

### Finalize Dataset

In [ ]:
# Define required columns
required_columns = [
    'caseid', 'villageyear', 'age_group', 'education_bucket', 'farmer_status', 'year',
    'invHHtvf1', 'invHHtvf2', 'madult', 'fadult', 'kids',
    'maleh', 'farm', 'ageh', 'age2h', 'educh', 'vfst', 'newst'
]

# Filter necessary columns
data = df[required_columns].copy()

# Convert 'villageyear' to numeric
data['villageyear'] = pd.to_numeric(data['villageyear'], errors='coerce')

# One-hot encoding for categorical variables
data = pd.get_dummies(data, columns=['age_group', 'education_bucket', 'farmer_status'], drop_first=True)

# Create year dummies
data = pd.concat([data, pd.get_dummies(data['year'], prefix='year', drop_first=True)], axis=1)
data.drop(columns=['year'], inplace=True)

# Rename dummy variables
rename_columns = {
    'age_group_Young': 'Young',
    'education_bucket_Medium (5-8 years)': 'Medium',
    'education_bucket_High (9-16 years)': 'High',
    'farmer_status_Non-Farmer': 'NonFarmer'
}
data.rename(columns=rename_columns, inplace=True)

# Convert dummy variables to integers
dummy_columns = ['Young', 'Medium', 'High', 'NonFarmer'] + [col for col in data.columns if col.startswith('year_')]
data[dummy_columns] = data[dummy_columns].astype(int)

# Drop rows with missing values
data.dropna(inplace=True)

# Set multi-index
data['caseid'] = data['caseid'].astype('category')
data['villageyear'] = data['villageyear'].astype(int)
data = data.set_index(['caseid', 'villageyear'])

## IV 2SLS Regression

### First Stage Regression (IV)

In [ ]:
# Define the first-stage regression formula
formula = """
vfst ~ invHHtvf1 + invHHtvf2 + madult + fadult + kids + maleh + farm + 
       ageh + age2h + educh + year_2 + year_3 + year_4 + year_5 + year_6 + year_7 + EntityEffects
"""

# Extract 'villageyear' for clustering
clusters = data.index.get_level_values('villageyear').to_series(index=data.index)

# Fit the PanelOLS model
model = PanelOLS.from_formula(formula, data)
firststage = model.fit(cov_type='clustered', clusters=clusters)

# Add predicted values to the dataset
data = data.copy()
data['vfst_hat'] = firststage.fitted_values

# Print regression summary
print(firststage.summary)

### Second Stage Regression

In [ ]:
# Create interaction terms
data['vfst_hat_Young'] = data['vfst_hat'] * data['Young']
data['vfst_hat_Medium'] = data['vfst_hat'] * data['Medium']
data['vfst_hat_High'] = data['vfst_hat'] * data['High']
data['vfst_hat_NonFarmer'] = data['vfst_hat'] * data['NonFarmer']

# Extract 'villageyear' as clusters
clusters = data.index.get_level_values('villageyear').to_series(index=data.index)

# Define regression formula
formula = """
newst ~ vfst_hat_Young + vfst_hat_Medium + vfst_hat_High + vfst_hat_NonFarmer
        + Young + Medium + High + NonFarmer
        + madult + fadult + kids + maleh
        + year_2 + year_3 + year_4 + year_5 + year_6 + year_7 + EntityEffects
"""

# Fit regression model with clustered standard errors
model = PanelOLS.from_formula(formula, data, drop_absorbed=True)
secondstage = model.fit(cov_type='clustered', clusters=clusters)

# Print results
print(secondstage.summary)

### OLS Regression

In [ ]:
# Create interaction terms for vfst and categorical groups
data['vfst_Young'] = data['vfst'] * data['Young']
data['vfst_Medium'] = data['vfst'] * data['Medium']
data['vfst_High'] = data['vfst'] * data['High']
data['vfst_NonFarmer'] = data['vfst'] * data['NonFarmer']

# Extract 'villageyear' as clusters
clusters = data.index.get_level_values('villageyear').to_series(index=data.index)

# Define OLS regression formula
formula = """
newst ~ vfst_Young + vfst_Medium + vfst_High + vfst_NonFarmer
        + Young + Medium + High + NonFarmer
        + madult + fadult + kids + maleh
        + year_2 + year_3 + year_4 + year_5 + year_6 + year_7 + EntityEffects
"""

# Fit OLS model with clustered standard errors
model = PanelOLS.from_formula(formula, data, drop_absorbed=True)
ols = model.fit(cov_type='clustered', clusters=clusters)

# Print results 
print(ols.summary)

## Durbin–Wu–Hausman Test

In [ ]:
# First-stage regression (IV)
formula_first_stage = """
vfst ~ invHHtvf1 + invHHtvf2 + madult + fadult + kids + maleh + farm +
        ageh + age2h + educh + year_2 + year_3 + year_4 + year_5 + year_6 + year_7 + EntityEffects
"""
clusters = data.index.get_level_values('villageyear').to_series(index=data.index)
model_first_stage = PanelOLS.from_formula(formula_first_stage, data)
result_first_stage = model_first_stage.fit(cov_type='clustered', clusters=clusters)
data['residuals_iv'] = result_first_stage.resids

# Second-stage OLS regression with residuals
formula_hausman = """
newst ~ vfst + residuals_iv + Young + Medium + High + NonFarmer
        + madult + fadult + kids + maleh 
        + year_2 + year_3 + year_4 + year_5 + year_6 + year_7 + EntityEffects
"""
model_hausman = PanelOLS.from_formula(formula_hausman, data, drop_absorbed=True)
result_hausman = model_hausman.fit(cov_type='clustered', clusters=clusters)

# Hausman test
try:
    coeff_residuals = result_hausman.params['residuals_iv']
    std_err_residuals = result_hausman.std_errors['residuals_iv']
    hausman_stat = (coeff_residuals / std_err_residuals) ** 2
    p_value = 1 - chi2.cdf(hausman_stat, df=1)

    print("Hausman Test for Endogeneity:")
    print(f"Hausman Statistic: {hausman_stat:.4f}")
    print(f"P-value: {p_value:.4f}")

    if p_value < 0.05:
        print("Reject the null hypothesis: Endogeneity detected. IV is necessary.")
    else:
        print("Fail to reject the null hypothesis: No endogeneity detected. OLS is consistent.")
except KeyError:
    print("Hausman test could not be computed: 'residuals_iv' coefficient not found.")

# First-stage F-statistic
print("\nFirst-stage F-statistic:")
print(result_first_stage.f_statistic)

## DAG Models

In [ ]:
def create_dag_iv_2sls():
    dag = Digraph(comment="IV 2SLS Regression DAG", format='png')
    dag.node('IV', 'Inverse Village Size')
    dag.node('PVFC', 'Predicted Village Fund Credit')
    dag.node('Interaction', 'Predicted Village Fund Credit x Subgroup')
    dag.node('NSC', 'New Short-Term Credit')
    dag.node('Controls', 'Control Variables')
    dag.edge('IV', 'PVFC', label=" Instrument")
    dag.edge('PVFC', 'Interaction', label=" Predicted Values")
    dag.edge('Interaction', 'NSC', xlabel="Interaction Effect        ")
    dag.edge('Controls', 'NSC', label="Controls")
    return dag

def create_dag_ols():
    dag = Digraph(comment="OLS Regression DAG", format='png')
    dag.node('VFC', 'Village Fund Credit')
    dag.node('Interaction', 'Village Fund Credit x Subgroup')
    dag.node('NSC', 'New Short-Term Credit')
    dag.node('Controls', 'Control Variables')
    dag.edge('VFC', 'Interaction', label=" Direct Effect")
    dag.edge('Interaction', 'NSC', xlabel="Interaction Effect         ")
    dag.edge('Controls', 'NSC', label="Controls")
    return dag

# Render DAGs
dag_iv = create_dag_iv_2sls()
dag_ols = create_dag_ols()

dag_iv.render(filename='iv_2sls_dag', cleanup=True)
dag_ols.render(filename='ols_dag', cleanup=True)

## Variable Selection

### Decision Tree

In [ ]:
# Prepare feature matrix with renamed variables for clarity
X = data[['vfst', 'madult', 'fadult', 'kids', 'maleh', 'farm', 'ageh', 'educh'] + 
         [col for col in data.columns if col.startswith('year_')]].copy()

X.rename(columns={
    'vfst': 'Village Fund Credit',
    'madult': 'Number of Male Adults in Household',
    'fadult': 'Number of Female Adults in Household',
    'kids': 'Number of Kids in Household',
    'maleh': 'Male Head of Household Dummy',
    'ageh': 'Head of Household\'s Age',
    'educh': 'Head of Household\'s Years of Education',
    'farm': 'Occupation (Farmer vs. Non-Farmer)',
    'year_2': 'Year 2',
    'year_3': 'Year 3',
    'year_4': 'Year 4',
    'year_5': 'Year 5',
    'year_6': 'Year 6',
    'year_7': 'Year 7',
}, inplace=True)

# Define target variable
y = data['newst']

# Fit DecisionTreeRegressor
tree = DecisionTreeRegressor(max_depth=3, random_state=0)
tree.fit(X, y)

# Predictions and evaluation
y_pred = tree.predict(X)
mse_tree = mean_squared_error(y, y_pred)
rmse_tree = np.sqrt(mse_tree)
r_squared_tree = r2_score(y, y_pred)

# Print metrics
print("Decision Tree Model Performance:")
print(f"Root Mean Squared Error (RMSE): {rmse_tree:.4f}")
print(f"R-squared: {r_squared_tree:.4f}")

# Plot decision tree
plt.figure(figsize=(24, 12))
plot_tree(
    tree,
    feature_names=X.columns,
    filled=True,
    rounded=True,
    fontsize=12,
    precision=2,
    impurity=False,
    proportion=True
)
plt.title("Decision Tree for Predicting New Short-term Credit Level", fontsize=16, fontweight='bold')
plt.show()

### Random Forest

In [ ]:
# Initialize and fit RandomForestRegressor
forest = RandomForestRegressor(n_estimators=100, max_depth=5, random_state=0)
forest.fit(X, y)

# Predictions and evaluation metrics
y_pred_forest = forest.predict(X)
mse_forest = mean_squared_error(y, y_pred_forest)
rmse_forest = np.sqrt(mse_forest)
r_squared_forest = r2_score(y, y_pred_forest)

# Print metrics
print(f"Random Forest - Mean Squared Error: {mse_forest}")
print(f"Random Forest - Root Mean Squared Error: {rmse_forest}")
print(f"Random Forest - R-squared: {r_squared_forest}")

# Feature importances
feature_importances = pd.DataFrame({
    'Feature': X.columns,
    'Importance': forest.feature_importances_
}).sort_values(by='Importance', ascending=False)

print("\nFeature Importance Matrix:")
print(feature_importances)

# Plot feature importances
plt.figure(figsize=(10, 6))
plt.barh(feature_importances['Feature'], feature_importances['Importance'])
plt.xlabel("Importance")
plt.ylabel("Feature")
plt.title("Feature Importance in Random Forest")
plt.gca().invert_yaxis()
plt.show()

### Boosting

In [ ]:
# Fit the GradientBoostingRegressor
boosting_model = GradientBoostingRegressor(n_estimators=100, max_depth=5, random_state=0)
boosting_model.fit(X, y)

# Make predictions
y_pred_boosting = boosting_model.predict(X)

# Evaluation metrics
mse_boosting = mean_squared_error(y, y_pred_boosting)
rmse_boosting = np.sqrt(mse_boosting)
r_squared_boosting = r2_score(y, y_pred_boosting)

print(f"Boosting - Mean Squared Error: {mse_boosting}")
print(f"Boosting - Root Mean Squared Error: {rmse_boosting}")
print(f"Boosting - R-squared: {r_squared_boosting}")

# Feature importance
feature_importances_boosting = pd.DataFrame({
    'Feature': X.columns,
    'Importance': boosting_model.feature_importances_
}).sort_values(by='Importance', ascending=False)

print("\nBoosting Feature Importance Matrix:")
print(feature_importances_boosting)

# Plot feature importance
plt.figure(figsize=(10, 6))
plt.barh(feature_importances_boosting['Feature'], feature_importances_boosting['Importance'])
plt.xlabel("Importance")
plt.ylabel("Feature")
plt.title("Feature Importance in Gradient Boosting")
plt.gca().invert_yaxis()
plt.show()

### Bagging

In [ ]:
# Initialize and fit BaggingRegressor
bagging_model = BaggingRegressor(
    base_estimator=DecisionTreeRegressor(max_depth=5),
    n_estimators=100,
    random_state=0
)
bagging_model.fit(X, y)

# Make predictions
y_pred_bagging = bagging_model.predict(X)

# Calculate evaluation metrics
mse_bagging = mean_squared_error(y, y_pred_bagging)
rmse_bagging = np.sqrt(mse_bagging)
r_squared_bagging = r2_score(y, y_pred_bagging)

# Print metrics
print(f"Bagging - Mean Squared Error: {mse_bagging}")
print(f"Bagging - Root Mean Squared Error: {rmse_bagging}")
print(f"Bagging - R-squared: {r_squared_bagging}")

# Calculate feature importances
feature_importances = np.mean([tree.feature_importances_ for tree in bagging_model.estimators_], axis=0)

# Create feature importance DataFrame
feature_importances_df = pd.DataFrame({
    'Feature': X.columns,
    'Importance': feature_importances
}).sort_values(by='Importance', ascending=False)

# Print feature importance matrix
print("\nBagging Feature Importance Matrix (Averaged from Base Estimators):")
print(feature_importances_df)

# Plot feature importances
plt.figure(figsize=(10, 6))
plt.barh(feature_importances_df['Feature'], feature_importances_df['Importance'])
plt.xlabel("Importance")
plt.ylabel("Feature")
plt.title("Feature Importance in Bagging")
plt.gca().invert_yaxis()
plt.show()

### Performance Evaluation 

In [ ]:
# Summarize error metrics for each model
error_comparison = pd.DataFrame({
    'Model': ['Decision Tree', 'Random Forest', 'Boosting', 'Bagging'],
    'MSE': [mse_tree, mse_forest, mse_boosting, mse_bagging],
    'RMSE': [rmse_tree, rmse_forest, rmse_boosting, rmse_bagging]
})

# Display the error comparison table
print("\nError Comparison Table:")
print(error_comparison)

### LASSO Regression

In [ ]:
# Standardize X and y
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
y_scaled = (y - y.mean()) / y.std()

# Fit LassoCV with cross-validation
lasso_cv = LassoCV(alphas=np.logspace(-4, 1, 50), cv=5, random_state=0)
lasso_cv.fit(X_scaled, y_scaled)

# Cross-validated MSE vs. -log(alpha) plot
mse_path_mean = lasso_cv.mse_path_.mean(axis=1)
mse_path_std = lasso_cv.mse_path_.std(axis=1)
log_alphas_cv = -np.log(lasso_cv.alphas_)

plt.figure(figsize=(8, 6))
plt.errorbar(log_alphas_cv, mse_path_mean, yerr=mse_path_std, fmt='o-', ecolor='lightgray', capsize=3)
plt.xlabel(r"$-\log(\lambda)$")
plt.ylabel("Cross-validated MSE")
plt.title("Cross-validated MSE vs. -log(λ)")
plt.ylim(mse_path_mean.min() - 0.01, mse_path_mean.max() + 0.01)
plt.grid(True)
plt.show()

# Plot Lasso Coefficients Path
alphas_lasso, coefs_lasso, _ = lasso_path(X_scaled, y_scaled, alphas=np.logspace(-4, 1, 50))
log_alphas_path = -np.log(alphas_lasso)

plt.figure(figsize=(8, 6))
for i, feature in enumerate(X.columns):
    plt.plot(log_alphas_path, coefs_lasso[i], label=feature)
plt.xlabel(r"$-\log(\lambda)$")
plt.ylabel("Standardized coefficients")
plt.title("Lasso Coefficients Path")
plt.legend(loc="upper right", fontsize='small')
plt.grid(True)
plt.show()

In [ ]:
# Standardize X and y
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
y_scaled = (y - y.mean()) / y.std()

# Fit LassoCV with cross-validation
lasso_cv = LassoCV(alphas=np.logspace(-4, 1, 50), cv=5, random_state=0)
lasso_cv.fit(X_scaled, y_scaled)

# Extract non-zero coefficients and corresponding features
important_features = X.columns[lasso_cv.coef_ != 0]
important_coefficients = lasso_cv.coef_[lasso_cv.coef_ != 0]

# Create DataFrame for sorting and plotting
important_features_df = pd.DataFrame({
    'Feature': important_features,
    'Coefficient': important_coefficients
}).sort_values(by='Coefficient', ascending=False)

# Plot important features
plt.figure(figsize=(10, 6))
plt.barh(important_features_df['Feature'], important_features_df['Coefficient'])
plt.xlabel("Coefficient Value")
plt.ylabel("Feature")
plt.title("Important Features Identified by LASSO")
plt.gca().invert_yaxis()
plt.grid(True)
plt.show()

### Ridge Regression

In [ ]:
# Standardize X and y
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
y_scaled = (y - y.mean()) / y.std()

# Define a range of alphas for Ridge
alphas_ridge = np.logspace(-10, 10, 100)

# Perform cross-validation for each alpha
mse_mean, mse_std = [], []
for alpha in alphas_ridge:
    ridge = Ridge(alpha=alpha)
    neg_mse_scores = cross_val_score(ridge, X_scaled, y_scaled, cv=5, scoring='neg_mean_squared_error')
    mse_scores = -neg_mse_scores
    mse_mean.append(mse_scores.mean())
    mse_std.append(mse_scores.std())

# Convert to arrays for plotting
mse_mean, mse_std = np.array(mse_mean), np.array(mse_std)
log_alphas = -np.log(alphas_ridge)

# Cross-validated MSE vs -log(alpha)
plt.figure(figsize=(8, 6))
plt.errorbar(log_alphas, mse_mean, yerr=mse_std, fmt='o-', ecolor='lightgray', capsize=3)
plt.xlabel(r"$-\log(\lambda)$")
plt.ylabel("Cross-validated MSE")
plt.title("Cross-validated MSE vs. -log(λ) for Ridge")
plt.ylim(mse_mean.min() - 0.0005, mse_mean.max() + 0.0005)
plt.grid(True)
plt.show()

# Ridge Coefficients Path
coefs_ridge = []
for alpha in alphas_ridge:
    ridge = Ridge(alpha=alpha)
    ridge.fit(X_scaled, y_scaled)
    coefs_ridge.append(ridge.coef_)

log_alphas_path = -np.log(alphas_ridge)
plt.figure(figsize=(10, 6))
for i, feature in enumerate(X.columns):
    plt.plot(log_alphas_path, [coef[i] for coef in coefs_ridge], label=feature)
plt.xlabel(r"$-\log(\lambda)$")
plt.ylabel("Coefficient Value")
plt.title("Ridge Coefficients Path")
plt.legend(loc="upper right", fontsize='small')
plt.xlim(log_alphas_path.min() - 2, log_alphas_path.max() + 2)
plt.grid(True)
plt.show()

# Important Features Plot
best_alpha = alphas_ridge[np.argmin(mse_mean)]
ridge_best = Ridge(alpha=best_alpha)
ridge_best.fit(X_scaled, y_scaled)

important_features_df = pd.DataFrame({
    'Feature': X.columns,
    'Coefficient': ridge_best.coef_
}).sort_values(by='Coefficient', ascending=False)

plt.figure(figsize=(10, 6))
plt.barh(important_features_df['Feature'], important_features_df['Coefficient'])
plt.xlabel("Coefficient Value")
plt.ylabel("Feature")
plt.title(f"Feature Importance in Ridge Regression (Best λ = {best_alpha})")
plt.gca().invert_yaxis()
plt.grid(True)
plt.show()

### DML

In [ ]:
# Prepare Data
data = data.copy()
X = data[['invHHtvf1', 'invHHtvf2', 'madult', 'fadult', 'kids', 'maleh', 'farm',
          'ageh', 'age2h', 'educh'] + [col for col in data.columns if col.startswith('year_')]]
T = data['vfst']  # Treatment
Y = data['newst']  # Outcome
groups = ['Young', 'Medium', 'High', 'NonFarmer']
residual_Y = np.zeros(len(Y))
residual_T = np.zeros(len(T))

# Cross-Fitting Residuals Using Machine Learning Models
kf = KFold(n_splits=5, shuffle=True, random_state=42)
param_grid = {
    'n_estimators': [50, 100, 150],
    'max_depth': [3, 5, 7, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

for train_idx, test_idx in kf.split(X):
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    Y_train, Y_test = Y.iloc[train_idx], Y.iloc[test_idx]
    T_train, T_test = T.iloc[train_idx], T.iloc[test_idx]

    # Hyperparameter tuning and fitting models
    grid_y = GridSearchCV(RandomForestRegressor(random_state=42), param_grid, cv=3, n_jobs=-1)
    grid_y.fit(X_train, Y_train)
    best_y = grid_y.best_estimator_

    grid_t = GridSearchCV(RandomForestRegressor(random_state=42), param_grid, cv=3, n_jobs=-1)
    grid_t.fit(X_train, T_train)
    best_t = grid_t.best_estimator_

    residual_Y[test_idx] = Y_test - best_y.predict(X_test)
    residual_T[test_idx] = T_test - best_t.predict(X_test)

# Prepare Interaction Terms
residual_T_df = pd.DataFrame({'residual_T': residual_T}, index=data.index)
for group in groups:
    residual_T_df[f'residual_T_{group}'] = residual_T_df['residual_T'] * data[group].values

# Second Stage Regression
interaction_cols = ['residual_T_' + g for g in groups]
second_stage_X = residual_T_df[['residual_T'] + interaction_cols]
second_stage_X = sm.add_constant(second_stage_X)
ols_model = sm.OLS(residual_Y, second_stage_X).fit()

# Display Results
print("Second Stage Coefficients (Double Machine Learning):")
print(ols_model.summary())

### Deep IV

In [ ]:
# Prepare data
data = data.copy()
Z = data[['invHHtvf1', 'invHHtvf2']].values
X = data[['madult', 'fadult', 'kids', 'maleh', 'farm', 'ageh', 'age2h', 'educh'] +
         [col for col in data.columns if col.startswith('year_')]].values
T = data['vfst'].values.reshape(-1, 1)
Y = data['newst'].values.reshape(-1, 1)

# Standardize covariates and instruments
scaler_X, scaler_Z = StandardScaler(), StandardScaler()
X_scaled, Z_scaled = scaler_X.fit_transform(X), scaler_Z.fit_transform(Z)

# Split data
X_train, X_test, Z_train, Z_test, T_train, T_test, Y_train, Y_test = train_test_split(
    X_scaled, Z_scaled, T, Y, test_size=0.2, random_state=42
)

# Treatment and outcome models
def create_treatment_model(input_shape):
    model = Sequential([
        Input(shape=(input_shape,)),
        Dense(64, activation='relu'),
        Dense(32, activation='relu'),
        Dense(1)
    ])
    model.compile(optimizer='adam', loss='mse')
    return model

def create_outcome_model(input_shape):
    model = Sequential([
        Input(shape=(input_shape,)),
        Dense(64, activation='relu'),
        Dense(32, activation='relu'),
        Dense(1)
    ])
    model.compile(optimizer='adam', loss='mse')
    return model

# Calculate treatment effects
def calculate_treatment_effects(X_test, outcome_model):
    T0 = np.zeros((X_test.shape[0], 1))
    T1 = np.ones((X_test.shape[0], 1))
    Y_T0 = outcome_model.predict(np.hstack([X_test, T0]), verbose=0)
    Y_T1 = outcome_model.predict(np.hstack([X_test, T1]), verbose=0)
    return np.mean(Y_T1 - Y_T0), Y_T1 - Y_T0

# Bootstrapping
n_bootstraps = 1
overall_effects = []
group_effects = {group: [] for group in ['Young', 'Medium', 'High', 'NonFarmer']}

for b in range(n_bootstraps):
    indices = np.random.choice(range(X_train.shape[0]), size=X_train.shape[0], replace=True)
    X_resampled, Z_resampled = X_train[indices], Z_train[indices]
    T_resampled, Y_resampled = T_train[indices], Y_train[indices]

    treatment_model = create_treatment_model(X_resampled.shape[1] + Z_resampled.shape[1])
    treatment_model.fit(
        np.hstack([Z_resampled, X_resampled]), T_resampled,
        epochs=10, batch_size=32, verbose=0,
        validation_split=0.1, callbacks=[EarlyStopping(patience=2, restore_best_weights=True)]
    )
    T_hat_resampled = treatment_model.predict(np.hstack([Z_resampled, X_resampled]), verbose=0)

    outcome_model = create_outcome_model(X_resampled.shape[1] + 1)
    outcome_model.fit(
        np.hstack([X_resampled, T_hat_resampled]), Y_resampled,
        epochs=10, batch_size=32, verbose=0,
        validation_split=0.1, callbacks=[EarlyStopping(patience=2, restore_best_weights=True)]
    )

    overall_effect, treatment_diff = calculate_treatment_effects(X_test, outcome_model)
    overall_effects.append(overall_effect)

    for group in ['Young', 'Medium', 'High', 'NonFarmer']:
        subgroup_indicator = data[group].iloc[:len(X_test)].values.reshape(-1, 1)
        subgroup_effect = np.mean(treatment_diff * subgroup_indicator)
        group_effects[group].append(subgroup_effect)

# Confidence intervals
def confidence_interval(effects):
    lower, upper = np.percentile(effects, 2.5), np.percentile(effects, 97.5)
    mean_effect = np.mean(effects)
    return mean_effect, lower, upper

mean_overall, lower_overall, upper_overall = confidence_interval(overall_effects)
print("\nDeep IV Estimated Treatment Effects with 95% Confidence Intervals:")
print(f"Overall Treatment Effect: {mean_overall:.4f} (95% CI: [{lower_overall:.4f}, {upper_overall:.4f}])")

for group in ['Young', 'Medium', 'High', 'NonFarmer']:
    mean_group, lower_group, upper_group = confidence_interval(group_effects[group])
    print(f"{group} Treatment Effect: {mean_group:.4f} (95% CI: [{lower_group:.4f}, {upper_group:.4f}])")

### Binary Treatment Creation

In [ ]:
# Create binary treatment variable
data['vfst_binary'] = np.where(data['vfst'] > 0, 1, 0)

# Verify treatment variable distribution
print("Distribution of Binary Treatment Variable (vfst_binary):")
print(data['vfst_binary'].value_counts())

# Create interaction terms for treatment and categorical groups
data['vfst_binary_Young'] = data['vfst_binary'] * data['Young']
data['vfst_binary_Medium'] = data['vfst_binary'] * data['Medium']
data['vfst_binary_High'] = data['vfst_binary'] * data['High']
data['vfst_binary_NonFarmer'] = data['vfst_binary'] * data['NonFarmer']

# Extract 'villageyear' as clusters
clusters = data.index.get_level_values('villageyear').to_series(index=data.index)

# Define regression formula
formula_binary = """
newst ~ vfst_binary_Young + vfst_binary_Medium + vfst_binary_High + vfst_binary_NonFarmer
         + Young + Medium + High + NonFarmer
         + madult + fadult + kids + maleh
         + year_2 + year_3 + year_4 + year_5 + year_6 + year_7 + EntityEffects
"""

# Display first few rows with interaction terms
print("First few rows with binary treatment and interaction terms:")
print(data[['vfst_binary', 'vfst_binary_Young', 'vfst_binary_Medium', 'vfst_binary_High', 'vfst_binary_NonFarmer']].head())

### Matching

In [ ]:
# Define covariates for PSM
covariates = ['madult', 'fadult', 'kids', 'maleh', 'ageh', 'educh', 'farm'] + \
             [col for col in data.columns if col.startswith('year_')]
X = data[covariates]
y_treatment = data['vfst_binary']
y_outcome = data['newst']

# Estimate Propensity Scores
logit = LogisticRegression(solver='lbfgs', max_iter=500)
logit.fit(X, y_treatment)
data['pscore'] = logit.predict_proba(X)[:, 1]

# Nearest-Neighbor Matching
treated = data[data['vfst_binary'] == 1]
control = data[data['vfst_binary'] == 0]
nn = NearestNeighbors(n_neighbors=1, metric='euclidean')
nn.fit(control[['pscore']])
distances, indices = nn.kneighbors(treated[['pscore']])

matched_control = control.iloc[indices.flatten()].reset_index(drop=True)
matched_treated = treated.reset_index(drop=True)
matched_data = pd.concat([matched_treated, matched_control], axis=0)

# Balance Check
def standardized_mean_diff(var, group1, group2):
    mean_diff = group1[var].mean() - group2[var].mean()
    pooled_std = np.sqrt((group1[var].var() + group2[var].var()) / 2)
    return mean_diff / pooled_std

pre_match_smd = {var: standardized_mean_diff(var, treated, control) for var in covariates}
post_match_smd = {var: standardized_mean_diff(var, matched_treated, matched_control) for var in covariates}

# Estimate Overall ATE and Subgroup ATEs with Bootstrapping
n_bootstraps = 1000
ate_bootstrap = []
subgroup_ates_bootstrap = {'Young': [], 'Medium': [], 'High': [], 'NonFarmer': []}

def estimate_ate(treated_df, control_df):
    return treated_df['newst'].mean() - control_df['newst'].mean()

def estimate_subgroup_ate(subgroup_var, subgroup_value, treated_df, control_df):
    treated_subgroup = treated_df[treated_df[subgroup_var] == subgroup_value]
    control_subgroup = control_df[control_df[subgroup_var] == subgroup_value]
    if len(treated_subgroup) > 0 and len(control_subgroup) > 0:
        return treated_subgroup['newst'].mean() - control_subgroup['newst'].mean()
    return np.nan

for i in range(n_bootstraps):
    boot_treated = matched_treated.sample(frac=1, replace=True, random_state=i)
    boot_control = matched_control.sample(frac=1, replace=True, random_state=i)
    ate_bootstrap.append(estimate_ate(boot_treated, boot_control))
    for subgroup in ['Young', 'Medium', 'High', 'NonFarmer']:
        subgroup_ates_bootstrap[subgroup].append(
            estimate_subgroup_ate(subgroup, 1, boot_treated, boot_control)
        )

def calculate_ci(boot_estimates):
    return np.percentile(boot_estimates, [2.5, 97.5])

# Report Results
ate_mean = np.mean(ate_bootstrap)
ate_ci = calculate_ci(ate_bootstrap)
print(f"Overall ATE: {ate_mean:.4f} (95% CI: {ate_ci[0]:.4f}, {ate_ci[1]:.4f})")

for subgroup in ['Young', 'Medium', 'High', 'NonFarmer']:
    subgroup_mean = np.mean(subgroup_ates_bootstrap[subgroup])
    subgroup_ci = calculate_ci(subgroup_ates_bootstrap[subgroup])
    print(f"ATE for {subgroup}: {subgroup_mean:.4f} (95% CI: {subgroup_ci[0]:.4f}, {subgroup_ci[1]:.4f})")

# Visualize Propensity Score Distribution
plt.figure(figsize=(10, 6))
plt.hist(treated['pscore'], bins=30, alpha=0.5, label='Treated', color='blue')
plt.hist(control['pscore'], bins=30, alpha=0.5, label='Control', color='orange')
plt.xlabel('Propensity Score')
plt.ylabel('Frequency')
plt.title('PSM Model: Propensity Score Distribution')
plt.legend()
plt.show()

### IPW

In [ ]:
# Define treatment and covariates
treatment = 'vfst_binary'
covariates = ['madult', 'fadult', 'kids', 'maleh', 'ageh', 'educh', 'farm',
              'year_2', 'year_3', 'year_4', 'year_5', 'year_6', 'year_7']

# Propensity score estimation using logistic regression
logit = LogisticRegression(max_iter=1000)
X = data[covariates]
y = data[treatment]
logit.fit(X, y)
data['pscore'] = logit.predict_proba(X)[:, 1]

# Calculate IPW weights
data['ipw'] = np.where(data[treatment] == 1, 1 / data['pscore'], 1 / (1 - data['pscore']))

# Calculate standardized mean difference
def standardized_mean_difference(var, treatment, weights=None):
    if weights is None:
        treated_mean = data.loc[data[treatment] == 1, var].mean()
        control_mean = data.loc[data[treatment] == 0, var].mean()
        pooled_std = np.sqrt((data.loc[data[treatment] == 1, var].var() +
                              data.loc[data[treatment] == 0, var].var()) / 2)
    else:
        treated_mean = (data.loc[data[treatment] == 1, var] * data.loc[data[treatment] == 1, weights]).sum() / \
                       data.loc[data[treatment] == 1, weights].sum()
        control_mean = (data.loc[data[treatment] == 0, var] * data.loc[data[treatment] == 0, weights]).sum() / \
                       data.loc[data[treatment] == 0, weights].sum()
        pooled_std = np.sqrt(0.5 * (data[var].std()**2))
    return (treated_mean - control_mean) / pooled_std

# Pre- and post-IPW balance
print("Standardized Mean Differences Pre-IPW:")
for cov in covariates:
    print(f"{cov}: {standardized_mean_difference(cov, treatment):.4f}")

print("\nStandardized Mean Differences Post-IPW:")
for cov in covariates:
    print(f"{cov}: {standardized_mean_difference(cov, treatment, 'ipw'):.4f}")

# Estimate ATE using weighted regression
def estimate_ate(data, treatment, outcome, weights):
    treated_outcome = (data[treatment] * data[outcome] * data[weights]).sum() / data[weights][data[treatment] == 1].sum()
    control_outcome = ((1 - data[treatment]) * data[outcome] * data[weights]).sum() / data[weights][data[treatment] == 0].sum()
    return treated_outcome - control_outcome

# Bootstrap ATE estimates
n_bootstraps = 1000
ate_bootstrap = []
subgroup_ates_bootstrap = {subgroup: [] for subgroup in ['Young', 'Medium', 'High', 'NonFarmer']}

for i in range(n_bootstraps):
    boot_data = data.sample(frac=1, replace=True, random_state=i)
    ate_bootstrap.append(estimate_ate(boot_data, 'vfst_binary', 'newst', 'ipw'))
    for subgroup in ['Young', 'Medium', 'High', 'NonFarmer']:
        subgroup_data = boot_data[boot_data[subgroup] == 1]
        if len(subgroup_data) > 0:
            subgroup_ates_bootstrap[subgroup].append(
                estimate_ate(subgroup_data, 'vfst_binary', 'newst', 'ipw')
            )

# Calculate confidence intervals
def calculate_ci(bootstrap_estimates):
    return np.percentile(bootstrap_estimates, [2.5, 97.5])

# Report results
overall_ate_mean = np.mean(ate_bootstrap)
overall_ate_ci = calculate_ci(ate_bootstrap)
print(f"\nOverall ATE: {overall_ate_mean:.4f} (95% CI: {overall_ate_ci[0]:.4f}, {overall_ate_ci[1]:.4f})")

for subgroup in ['Young', 'Medium', 'High', 'NonFarmer']:
    subgroup_mean = np.mean(subgroup_ates_bootstrap[subgroup])
    subgroup_ci = calculate_ci(subgroup_ates_bootstrap[subgroup])
    print(f"ATE for {subgroup}: {subgroup_mean:.4f} (95% CI: {subgroup_ci[0]:.4f}, {subgroup_ci[1]:.4f})")

# Visualize propensity score distribution
plt.figure(figsize=(10, 6))
plt.hist(data.loc[data[treatment] == 1, 'pscore'], bins=50, alpha=0.6, label='Treated', color='blue')
plt.hist(data.loc[data[treatment] == 0, 'pscore'], bins=50, alpha=0.6, label='Control', color='orange')
plt.xlabel('Propensity Score')
plt.ylabel('Frequency')
plt.title('IPW Model: Propensity Score Distribution')
plt.legend()
plt.show()

### MetaLearners

In [ ]:
# Suppress warnings and ensure reproducibility
warnings.filterwarnings("ignore", category=UserWarning)
logging.getLogger("lightgbm").setLevel(logging.ERROR)
np.random.seed(123)

# Prepare data
data = data.copy()
data['vfst_binary'] = np.where(data['vfst'] > 0, 1, 0)

# Define variables
X = ['madult', 'fadult', 'kids', 'maleh', 'ageh', 'educh', 'farm', 
     'year_2', 'year_3', 'year_4', 'year_5', 'year_6', 'year_7']
T = 'vfst_binary'
Y = 'newst'
subgroups = ['Young', 'Medium', 'High', 'NonFarmer']

# Bootstrap function for 95% CIs
def bootstrap_ate(learner_func, n_bootstrap=1):
    ate_bootstrap = []
    for _ in range(n_bootstrap):
        sample_data = data.sample(frac=1, replace=True, random_state=np.random.randint(1, 1_000_000))
        ate_bootstrap.append(learner_func(sample_data))
    lower = np.percentile(ate_bootstrap, 2.5)
    upper = np.percentile(ate_bootstrap, 97.5)
    return np.mean(ate_bootstrap), (lower, upper)

# Helper function to train model
def train_model(X_train, y_train, **kwargs):
    model = LGBMRegressor(n_estimators=100, max_depth=5, verbose=-1, **kwargs)
    model.fit(X_train, y_train)
    return model

# S-Learner
def s_learner_ate(sample):
    s_learner = train_model(sample[X + [T]], sample[Y])
    s_cate = (s_learner.predict(sample[X].assign(**{T: 1})) - 
              s_learner.predict(sample[X].assign(**{T: 0})))
    return np.mean(s_cate)

mean_ate, ci = bootstrap_ate(s_learner_ate)
print(f"\n--- S-Learner ---\nOverall ATE (S-Learner): {mean_ate:.4f} (95% CI: {ci})")

for subgroup in subgroups:
    def s_learner_subgroup(sample):
        subgroup_data = sample[sample[subgroup] == 1]
        s_learner = train_model(subgroup_data[X + [T]], subgroup_data[Y])
        s_cate_subgroup = (s_learner.predict(subgroup_data[X].assign(**{T: 1})) - 
                           s_learner.predict(subgroup_data[X].assign(**{T: 0})))
        return np.mean(s_cate_subgroup)
    
    mean_ate, ci = bootstrap_ate(s_learner_subgroup)
    print(f"ATE for {subgroup} (S-Learner): {mean_ate:.4f} (95% CI: {ci})")

# T-Learner
def t_learner_ate(sample):
    model_control = train_model(sample[sample[T] == 0][X], sample[sample[T] == 0][Y])
    model_treated = train_model(sample[sample[T] == 1][X], sample[sample[T] == 1][Y])
    t_cate = model_treated.predict(sample[X]) - model_control.predict(sample[X])
    return np.mean(t_cate)

mean_ate, ci = bootstrap_ate(t_learner_ate)
print(f"\n--- T-Learner ---\nOverall ATE (T-Learner): {mean_ate:.4f} (95% CI: {ci})")

for subgroup in subgroups:
    def t_learner_subgroup(sample):
        subgroup_data = sample[sample[subgroup] == 1]
        model_control = train_model(subgroup_data[subgroup_data[T] == 0][X], subgroup_data[subgroup_data[T] == 0][Y])
        model_treated = train_model(subgroup_data[subgroup_data[T] == 1][X], subgroup_data[subgroup_data[T] == 1][Y])
        t_cate = model_treated.predict(subgroup_data[X]) - model_control.predict(subgroup_data[X])
        return np.mean(t_cate)
    
    mean_ate, ci = bootstrap_ate(t_learner_subgroup)
    print(f"ATE for {subgroup} (T-Learner): {mean_ate:.4f} (95% CI: {ci})")

# X-Learner
def x_learner_ate(sample):
    model_control = train_model(sample[sample[T] == 0][X], sample[sample[T] == 0][Y])
    model_treated = train_model(sample[sample[T] == 1][X], sample[sample[T] == 1][Y])
    tau_control = model_treated.predict(sample[sample[T] == 0][X]) - sample[sample[T] == 0][Y]
    tau_treated = sample[sample[T] == 1][Y] - model_control.predict(sample[sample[T] == 1][X])
    model_tau_control = train_model(sample[sample[T] == 0][X], tau_control)
    model_tau_treated = train_model(sample[sample[T] == 1][X], tau_treated)
    logit = LogisticRegression(max_iter=500)
    logit.fit(sample[X], sample[T])
    pscore = logit.predict_proba(sample[X])[:, 1]
    x_cate = (pscore * model_tau_control.predict(sample[X]) +
              (1 - pscore) * model_tau_treated.predict(sample[X]))
    return np.mean(x_cate)

mean_ate, ci = bootstrap_ate(x_learner_ate)
print(f"\n--- X-Learner ---\nOverall ATE (X-Learner): {mean_ate:.4f} (95% CI: {ci})")

for subgroup in subgroups:
    def x_learner_subgroup(sample):
        subgroup_data = sample[sample[subgroup] == 1]
        return x_learner_ate(subgroup_data)
    
    mean_ate, ci = bootstrap_ate(x_learner_subgroup)
    print(f"ATE for {subgroup} (X-Learner): {mean_ate:.4f} (95% CI: {ci})")

### Doubly Robust

In [ ]:
# Suppress warnings and ensure reproducibility
warnings.filterwarnings("ignore", category=UserWarning)
logging.getLogger("lightgbm").setLevel(logging.ERROR)
np.random.seed(123)

# Prepare data
data = data.copy()
data['vfst_binary'] = np.where(data['vfst'] > 0, 1, 0)

# Define variables
X = ['madult', 'fadult', 'kids', 'maleh', 'ageh', 'educh', 'farm', 
     'year_2', 'year_3', 'year_4', 'year_5', 'year_6', 'year_7']
T = 'vfst_binary'
Y = 'newst'
subgroups = ['Young', 'Medium', 'High', 'NonFarmer']

# Doubly Robust Estimator
def doubly_robust(df, X, T, Y):
    ps = LogisticRegression(max_iter=500, solver='lbfgs').fit(df[X], df[T]).predict_proba(df[X])[:, 1]
    model_control = LGBMRegressor(n_estimators=100, max_depth=5, min_data_in_leaf=10, min_gain_to_split=0.01)
    model_treated = LGBMRegressor(n_estimators=100, max_depth=5, min_data_in_leaf=10, min_gain_to_split=0.01)
    model_control.fit(df[df[T] == 0][X], df[df[T] == 0][Y])
    model_treated.fit(df[df[T] == 1][X], df[df[T] == 1][Y])
    mu0 = model_control.predict(df[X])
    mu1 = model_treated.predict(df[X])
    dr_ate = (
        np.mean(df[T] * (df[Y] - mu1) / ps + mu1) -
        np.mean((1 - df[T]) * (df[Y] - mu0) / (1 - ps) + mu0)
    )
    return dr_ate

# Bootstrap for Confidence Intervals
def bootstrap_ate(n_bootstrap=1):
    ate_bootstrap = []
    for _ in range(n_bootstrap):
        sample_data = data.sample(frac=1, replace=True, random_state=np.random.randint(0, 10000))
        ate_bootstrap.append(doubly_robust(sample_data, X, T, Y))
    lower = np.percentile(ate_bootstrap, 2.5)
    upper = np.percentile(ate_bootstrap, 97.5)
    return np.mean(ate_bootstrap), (lower, upper)

# Overall ATE
mean_ate, ci = bootstrap_ate()
print(f"Overall ATE (Doubly Robust): {mean_ate:.4f} (95% CI: {ci})")

# Subgroup ATEs
for subgroup in subgroups:
    def doubly_robust_subgroup(sample):
        subgroup_data = sample[sample[subgroup] == 1]
        return doubly_robust(subgroup_data, X, T, Y)
    mean_ate, ci = bootstrap_ate()
    print(f"ATE for {subgroup} (Doubly Robust): {mean_ate:.4f} (95% CI: {ci})")

### Causal Forests

#### Binary Treatment

In [ ]:
# Prepare data
Y = data['newst'].values
T = data['vfst_binary'].values
X = data[['Young', 'Medium', 'High', 'NonFarmer', 
          'madult', 'fadult', 'kids', 'maleh', 
          'year_2', 'year_3', 'year_4', 'year_5', 'year_6', 'year_7']].values

# Standardize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Fit CausalForestDML
causal_forest = CausalForestDML(
    model_t=LassoCV(cv=5),
    model_y=LassoCV(cv=5),
    n_estimators=500,
    min_samples_leaf=10,
    max_depth=5,
    random_state=123
)
causal_forest.fit(Y=Y, T=T, W=X_scaled, X=X_scaled)

# Predict treatment effects
treatment_effects = causal_forest.effect(X_scaled)
data['treatment_effects'] = treatment_effects

# Summarize treatment effects by subgroup
print("Summary of Treatment Effects by Subgroup:")
for subgroup in ['Young', 'Medium', 'High', 'NonFarmer']:
    subgroup_effects = data.loc[data[subgroup] == 1, 'treatment_effects']
    print(f"\nSubgroup: {subgroup}")
    print(subgroup_effects.describe())

# Visualize treatment effect distribution
plt.figure(figsize=(10, 6))
for subgroup in ['Young', 'Medium', 'High', 'NonFarmer']:
    subgroup_effects = data.loc[data[subgroup] == 1, 'treatment_effects']
    plt.hist(subgroup_effects, bins=30, alpha=0.5, label=subgroup)

plt.title("Binary Treatment: Treatment Effect Distribution by Subgroup")
plt.xlabel("Treatment Effect")
plt.ylabel("Frequency")
plt.legend()
plt.show()

# Visualize feature importance
feature_importances = causal_forest.feature_importances_

plt.figure(figsize=(12, 6))
plt.barh(['Young', 'Medium', 'High', 'NonFarmer', 
          'madult', 'fadult', 'kids', 'maleh', 
          'year_2', 'year_3', 'year_4', 'year_5', 'year_6', 'year_7'],
         feature_importances)
plt.xlabel("Importance")
plt.ylabel("Features")
plt.title("Binary Treatment: Feature Importance from Causal Forest")
plt.show()

#### Unpredicted Treatment

In [ ]:
# Prepare data
Y = data['newst'].values  
T = data['vfst'].values 
X = data[['Young', 'Medium', 'High', 'NonFarmer', 
          'madult', 'fadult', 'kids', 'maleh', 
          'year_2', 'year_3', 'year_4', 'year_5', 'year_6', 'year_7']].values  # Features

# Standardize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Fit the CausalForestDML model
causal_forest = CausalForestDML(
    model_t=LassoCV(cv=5),
    model_y=LassoCV(cv=5),
    n_estimators=500,
    min_samples_leaf=10,
    max_depth=5,
    random_state=123,
)
causal_forest.fit(Y=Y, T=T, W=X_scaled, X=X_scaled)

# Predict treatment effects
treatment_effects = causal_forest.effect(X_scaled)
data['treatment_effects'] = treatment_effects

# Summarize treatment effects by subgroup
print("Summary of Treatment Effects by Subgroup:")
for subgroup in ['Young', 'Medium', 'High', 'NonFarmer']:
    subgroup_effects = data.loc[data[subgroup] == 1, 'treatment_effects']
    print(f"\nSubgroup: {subgroup}")
    print(subgroup_effects.describe())

# Visualize treatment effect distribution
plt.figure(figsize=(10, 6))
for subgroup in ['Young', 'Medium', 'High', 'NonFarmer']:
    subgroup_effects = data.loc[data[subgroup] == 1, 'treatment_effects']
    plt.hist(subgroup_effects, bins=30, alpha=0.5, label=subgroup)

plt.title("Treatment Effect Distribution by Subgroup")
plt.xlabel("Treatment Effect")
plt.ylabel("Frequency")
plt.legend()
plt.show()

# Feature importance visualization
feature_importances = causal_forest.feature_importances_

plt.figure(figsize=(12, 6))
plt.barh(['Young', 'Medium', 'High', 'NonFarmer', 
          'madult', 'fadult', 'kids', 'maleh', 
          'year_2', 'year_3', 'year_4', 'year_5', 'year_6', 'year_7'],
         feature_importances)
plt.xlabel("Importance")
plt.ylabel("Features")
plt.title("Feature Importance from Causal Forest")
plt.show()

#### Predicted Treatment

In [ ]:
# Prepare data
Y = data['newst'].values
T = data['vfst_hat'].values
X = data[['Young', 'Medium', 'High', 'NonFarmer', 
          'madult', 'fadult', 'kids', 'maleh', 
          'year_2', 'year_3', 'year_4', 'year_5', 'year_6', 'year_7']].values

# Standardize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Fit the CausalForestDML model
causal_forest = CausalForestDML(
    model_t=LassoCV(cv=5),
    model_y=LassoCV(cv=5),
    n_estimators=500,
    min_samples_leaf=10,
    max_depth=5,
    random_state=123,
)
causal_forest.fit(Y=Y, T=T, W=X_scaled, X=X_scaled)

# Predict treatment effects
treatment_effects = causal_forest.effect(X_scaled)
data['treatment_effects'] = treatment_effects

# Summarize treatment effects by subgroup
print("Summary of Treatment Effects by Subgroup:")
for subgroup in ['Young', 'Medium', 'High', 'NonFarmer']:
    subgroup_effects = data.loc[data[subgroup] == 1, 'treatment_effects']
    print(f"\nSubgroup: {subgroup}")
    print(subgroup_effects.describe())

# Visualize treatment effect distribution
plt.figure(figsize=(10, 6))
for subgroup in ['Young', 'Medium', 'High', 'NonFarmer']:
    subgroup_effects = data.loc[data[subgroup] == 1, 'treatment_effects']
    plt.hist(subgroup_effects, bins=30, alpha=0.5, label=subgroup)

plt.title("Treatment Effect Distribution by Subgroup")
plt.xlabel("Treatment Effect")
plt.ylabel("Frequency")
plt.legend()
plt.show()

# Feature importance visualization
feature_importances = causal_forest.feature_importances_

plt.figure(figsize=(12, 6))
plt.barh(['Young', 'Medium', 'High', 'NonFarmer', 
          'madult', 'fadult', 'kids', 'maleh', 
          'year_2', 'year_3', 'year_4', 'year_5', 'year_6', 'year_7'],
         feature_importances)
plt.xlabel("Importance")
plt.ylabel("Features")
plt.title("Feature Importance from Causal Forest")
plt.show()